# Web Scrapping using Beautiful Soup

## Amazon web Scrapping

In [40]:
#Import all required libraries

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [41]:
# Function to extract Product Title

def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price

def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-price-whole'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'class':'a-price aok-align-center reinventPricePriceToPayMargin priceToPay'}).find("span",attrs={'a-offscreen'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating

def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating


# Function to extract Number of User Reviews

def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count


# Function to extract Availability Status

def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

### First page web scrapping

In [42]:
# Add user agent 
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

# Add Webpage URL
url = "https://www.amazon.in/s?k=phones&page=2&crid=3NTCCNBP29QFT&qid=1682072571&sprefix=phones%2Caps%2C91&ref=sr_pg_1"

# HTTP Request
webpage = requests.get(url, headers=HEADERS)

# Soup Object containing all data
soup = BeautifulSoup(webpage.content, "html.parser")

# Fetch links as List of Tag Objects
links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

# Store the links
links_list = []

# Loop for extracting links from Tag Objects
for link in links:
        links_list.append(link.get('href'))

d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

# Loop for extracting product details from each link 
for link in links_list:
    new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)

    new_soup = BeautifulSoup(new_webpage.content, "html.parser")

    # Function calls to display all necessary product information
    d['title'].append(get_title(new_soup))
    d['price'].append(get_price(new_soup))
    d['rating'].append(get_rating(new_soup))
    d['reviews'].append(get_review_count(new_soup))
    d['availability'].append(get_availability(new_soup))

df = pd.DataFrame.from_dict(d)
df['title'].replace('', np.nan, inplace=True)
df = df.dropna(subset=['title'])

df

,title,price,rating,reviews,availability
0,"realme narzo N55 (Prime Blue, 4GB+64GB) 33W Se...","₹10,999",4.4 out of 5 stars,169 ratings,In stock
1,Apple iPhone 14 (128 GB) - Starlight,"₹71,900",4.5 out of 5 stars,531 ratings,In stock
2,"OnePlus Nord 2T 5G (Gray Shadow, 8GB RAM, 128G...","₹28,999",4.3 out of 5 stars,"24,441 ratings",In stock
3,"Samsung Galaxy M04 Dark Blue, 4GB RAM, 64GB St...","₹7,749",3.9 out of 5 stars,"4,020 ratings",In stock
4,"Oppo A17k (Blue, 3GB RAM, 64GB Storage) with N...","₹9,499",4.2 out of 5 stars,112 ratings,In stock
5,"Redmi Note 11 (Space Black, 4GB RAM, 64GB Stor...","₹12,999",4.1 out of 5 stars,"55,910 ratings",In stock
6,"Samsung Galaxy M13 (Midnight Blue, 6GB, 128GB ...","₹12,999",4.1 out of 5 stars,"25,859 ratings",In stock
7,"realme narzo 50 (Speed Black, 6GB RAM+128GB St...","₹13,499",4.2 out of 5 stars,"16,177 ratings",In stock
8,"Redmi 10 Power (Sporty Orange, 8GB RAM, 128GB ...","₹12,499",4.0 out of 5 stars,"1,295 ratings",In stock
9,"Oppo A78 5G (Glowing Black, 8GB RAM, 128 Stora...","₹18,999",4.1 out of 5 stars,960 ratings,In stock


### Multiple pages web scrapping

In [44]:
# Loop through pages from 2nd to 50th and add new data to initial df
for i in range(2, 50):
    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    url = f"https://www.amazon.in/s?k=phones&page=2&crid=3NTCCNBP29QFT&qid=1682072571&sprefix=phones%2Caps%2C91&ref=sr_pg_{i}"

    # HTTP Request
    webpage = requests.get(url, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))


    df_i = pd.DataFrame.from_dict(d)
    df_i['title'].replace('', np.nan, inplace=True)
    df_i = df_i.dropna(subset=['title'])

    df = pd.concat([df, df_i])


In [45]:
df

,title,price,rating,reviews,availability
0,"realme narzo N55 (Prime Blue, 4GB+64GB) 33W Se...","₹10,999",4.4 out of 5 stars,169 ratings,In stock
1,Apple iPhone 14 (128 GB) - Starlight,"₹71,900",4.5 out of 5 stars,531 ratings,In stock
2,"OnePlus Nord 2T 5G (Gray Shadow, 8GB RAM, 128G...","₹28,999",4.3 out of 5 stars,"24,441 ratings",In stock
3,"Samsung Galaxy M04 Dark Blue, 4GB RAM, 64GB St...","₹7,749",3.9 out of 5 stars,"4,020 ratings",In stock
4,"Oppo A17k (Blue, 3GB RAM, 64GB Storage) with N...","₹9,499",4.2 out of 5 stars,112 ratings,In stock
...,...,...,...,...,...
12,"Tecno Spark 9 (Sky Mirror, 4GB RAM,64GB Storag...","₹7,799",3.9 out of 5 stars,"4,825 ratings",In stock
21,"Samsung Galaxy M33 5G (Emerald Brown, 8GB, 128...","₹17,499",4.1 out of 5 stars,"27,309 ratings",In stock
6,"Redmi 10 Power (Sporty Orange, 8GB RAM, 128GB ...","₹12,499",4.0 out of 5 stars,"1,295 ratings",In stock
1,"Redmi A1 (Light Green, 2GB RAM 32GB ROM) | Seg...","₹5,699",4.0 out of 5 stars,"14,112 ratings",In stock


In [46]:
final_data = df

In [47]:
final_data

,title,price,rating,reviews,availability
0,"realme narzo N55 (Prime Blue, 4GB+64GB) 33W Se...","₹10,999",4.4 out of 5 stars,169 ratings,In stock
1,Apple iPhone 14 (128 GB) - Starlight,"₹71,900",4.5 out of 5 stars,531 ratings,In stock
2,"OnePlus Nord 2T 5G (Gray Shadow, 8GB RAM, 128G...","₹28,999",4.3 out of 5 stars,"24,441 ratings",In stock
3,"Samsung Galaxy M04 Dark Blue, 4GB RAM, 64GB St...","₹7,749",3.9 out of 5 stars,"4,020 ratings",In stock
4,"Oppo A17k (Blue, 3GB RAM, 64GB Storage) with N...","₹9,499",4.2 out of 5 stars,112 ratings,In stock
...,...,...,...,...,...
12,"Tecno Spark 9 (Sky Mirror, 4GB RAM,64GB Storag...","₹7,799",3.9 out of 5 stars,"4,825 ratings",In stock
21,"Samsung Galaxy M33 5G (Emerald Brown, 8GB, 128...","₹17,499",4.1 out of 5 stars,"27,309 ratings",In stock
6,"Redmi 10 Power (Sporty Orange, 8GB RAM, 128GB ...","₹12,499",4.0 out of 5 stars,"1,295 ratings",In stock
1,"Redmi A1 (Light Green, 2GB RAM 32GB ROM) | Seg...","₹5,699",4.0 out of 5 stars,"14,112 ratings",In stock


In [48]:
# Save the data to a CSV file

final_data.to_csv('Amazon.csv', index=False)

## Insert scrapped data into Mongodb

In [49]:
import pymongo
import json

In [50]:
client = pymongo.MongoClient("mongodb://localhost:27017/")


In [51]:
final_data.head()

,title,price,rating,reviews,availability
0,"realme narzo N55 (Prime Blue, 4GB+64GB) 33W Se...","₹10,999",4.4 out of 5 stars,169 ratings,In stock
1,Apple iPhone 14 (128 GB) - Starlight,"₹71,900",4.5 out of 5 stars,531 ratings,In stock
2,"OnePlus Nord 2T 5G (Gray Shadow, 8GB RAM, 128G...","₹28,999",4.3 out of 5 stars,"24,441 ratings",In stock
3,"Samsung Galaxy M04 Dark Blue, 4GB RAM, 64GB St...","₹7,749",3.9 out of 5 stars,"4,020 ratings",In stock
4,"Oppo A17k (Blue, 3GB RAM, 64GB Storage) with N...","₹9,499",4.2 out of 5 stars,112 ratings,In stock


In [52]:
# final_data.drop(['level_0', 'index'], axis=1, inplace=True)


In [55]:
final_data.shape

(926, 5)

In [56]:
dict = final_data.to_dict(orient = "records")

In [57]:
dict

[{'title': 'realme narzo N55 (Prime Blue, 4GB+64GB) 33W Segment Fastest Charging | Super High-res 64MP Primary AI Camera',
  'price': '₹10,999',
  'rating': '4.4 out of 5 stars',
  'reviews': '169 ratings',
  'availability': 'In stock'},
 {'title': 'Apple iPhone 14 (128 GB) - Starlight',
  'price': '₹71,900',
  'rating': '4.5 out of 5 stars',
  'reviews': '531 ratings',
  'availability': 'In stock'},
 {'title': 'OnePlus Nord 2T 5G (Gray Shadow, 8GB RAM, 128GB Storage)',
  'price': '₹28,999',
  'rating': '4.3 out of 5 stars',
  'reviews': '24,441 ratings',
  'availability': 'In stock'},
 {'title': 'Samsung Galaxy M04 Dark Blue, 4GB RAM, 64GB Storage | Upto 8GB RAM with RAM Plus | MediaTek Helio P35 Octa-core Processor | 5000 mAh Battery | 13MP Dual Camera',
  'price': '₹7,749',
  'rating': '3.9 out of 5 stars',
  'reviews': '4,020 ratings',
  'availability': 'In stock'},
 {'title': 'Oppo A17k (Blue, 3GB RAM, 64GB Storage) with No Cost EMI/Additional Exchange Offers',
  'price': '₹9,499'

In [58]:
db = client["DAP_Project"]

In [59]:
print(db)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'DAP_Project')


In [60]:
db.Amazon.insert_many(dict)

## Flipkart web Scrapping

In [35]:
# The functions are different for each website

# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"class":'B_NuCI'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("div", attrs={'class':'_30jeq3 _16Jk6d'}).string.strip()

    except:
        price = ""

    return price

# Function to extract Camera Rating
def get_camera_rating(soup1):

    try:
        camera_rating = soup.find("text", attrs={'class':'_2Ix0io'}).string.strip()

    except:
        camera_rating = ""

    return camera_rating

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("div", attrs={'class':'_2d4LTz'}).string.strip()

    except:
        rating = ""

    return rating

# Function to extract Number of User Ratings
def get_rating_count(soup):
    try:
        rating_count = soup.find("div", attrs={'class':'row _2afbiS'}).string.strip()
        
    except AttributeError:
        rating_count = ""

    return rating_count


### First page web scrapping

In [36]:
# add your user agent 
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

# The webpage URL
url = "https://www.flipkart.com/mobiles/pr?sid=tyy,4io&marketplace=FLIPKART"

# HTTP Request
webpage = requests.get(url, headers=HEADERS)

# Soup Object containing all data
soup = BeautifulSoup(webpage.content, "html.parser")

# Fetch links as List of Tag Objects
links = soup.find_all("a", attrs={'class':'_1fQZEK'})

# Store the links
links_list = []

# Loop for extracting links from Tag Objects
for link in links:
        links_list.append(link.get('href'))

d = {"title":[], "price":[], "rating":[], "camera_rating":[], "rating_count":[]}

# Loop for extracting product details from each link 
for link in links_list:
    new_webpage = requests.get("https://www.flipkart.com" + link, headers=HEADERS)

    new_soup = BeautifulSoup(new_webpage.content, "html.parser")

    # Function calls to display all necessary product information
    d['title'].append(get_title(new_soup))
    d['price'].append(get_price(new_soup))
    d['rating'].append(get_rating(new_soup))
    d['camera_rating'].append(get_camera_rating(new_soup))
    d['rating_count'].append(get_rating_count(new_soup))


df1 = pd.DataFrame.from_dict(d)
df1['title'].replace('', np.nan, inplace=True)
df1 = df1.dropna(subset=['title'])

df1

,title,price,rating,camera_rating,rating_count
0,"SAMSUNG Galaxy F04 (Opal Green, 64 GB) (4 GB ...","₹6,999",4.3,,"10,820 Ratings &"
1,"Infinix SMART 7 (Azure Blue, 64 GB) (4 GB RAM)","₹7,299",4.2,,"13,920 Ratings &"
2,"Infinix SMART 7 (Emerald Green, 64 GB) (4 GB ...","₹7,299",4.2,,"13,920 Ratings &"
3,"POCO C51 (Royal Blue, 64 GB) (4 GB RAM)","₹6,999",4.3,,"1,041 Ratings &"
4,"SAMSUNG Galaxy F13 (Nightsky Green, 64 GB) (4...","₹10,999",4.4,,"1,40,276 Ratings &"
5,"SAMSUNG Galaxy F13 (Waterfall Blue, 64 GB) (4...","₹10,999",4.4,,"1,40,276 Ratings &"
6,"SAMSUNG Galaxy F04 (Jade Purple, 64 GB) (4 GB...","₹6,999",4.3,,"10,820 Ratings &"
7,"Infinix SMART 7 (Night Black, 64 GB) (4 GB RAM)","₹7,299",4.2,,"13,920 Ratings &"
8,"POCO C51 (Power Black, 64 GB) (4 GB RAM)","₹6,999",4.3,,"1,041 Ratings &"
9,"POCO C55 (Cool Blue, 64 GB) (4 GB RAM)","₹7,999",4.2,,"10,256 Ratings &"


### Multiple pages web scrapping

In [61]:
# Loop through pages from 2nd to 50th and add new data to initial df
for i in range(2, 50):
    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 OPR/97.0.0.0', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    url = f"https://www.flipkart.com/search?sid=tyy%2C4io&otracker=CLP_Filters&page={i}"

    # HTTP Request
    webpage = requests.get(url, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'_1fQZEK'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "camera_rating":[], "rating_count":[]}

    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.flipkart.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['camera_rating'].append(get_camera_rating(new_soup))
        d['rating_count'].append(get_rating_count(new_soup))


    df_i = pd.DataFrame.from_dict(d)
    df_i['title'].replace('', np.nan, inplace=True)
    df_i = df_i.dropna(subset=['title'])

    df1 = pd.concat([df1, df_i])


In [62]:
df1

,title,price,rating,camera_rating,rating_count
0,"SAMSUNG Galaxy F04 (Opal Green, 64 GB) (4 GB ...","₹6,999",4.3,,"10,820 Ratings &"
1,"Infinix SMART 7 (Azure Blue, 64 GB) (4 GB RAM)","₹7,299",4.2,,"13,920 Ratings &"
2,"Infinix SMART 7 (Emerald Green, 64 GB) (4 GB ...","₹7,299",4.2,,"13,920 Ratings &"
3,"POCO C51 (Royal Blue, 64 GB) (4 GB RAM)","₹6,999",4.3,,"1,041 Ratings &"
4,"SAMSUNG Galaxy F13 (Nightsky Green, 64 GB) (4...","₹10,999",4.4,,"1,40,276 Ratings &"
...,...,...,...,...,...
19,GREENBERRI VIRAT (BLACK & BLUE),₹749,3.8,,507 Ratings &
20,"Infinix HOT 12 Play (Horizon Blue, 64 GB) (4 ...","₹8,999",4.3,,"1,15,557 Ratings &"
21,"Infinix HOT 12 Play (Racing Black, 64 GB) (4 ...","₹8,999",4.3,,"1,15,557 Ratings &"
22,"Infinix HOT 12 Play (Daylight Green, 64 GB) (...","₹8,999",4.3,,"1,15,557 Ratings &"


In [63]:
final_data1 = df1

In [64]:
final_data1

,title,price,rating,camera_rating,rating_count
0,"SAMSUNG Galaxy F04 (Opal Green, 64 GB) (4 GB ...","₹6,999",4.3,,"10,820 Ratings &"
1,"Infinix SMART 7 (Azure Blue, 64 GB) (4 GB RAM)","₹7,299",4.2,,"13,920 Ratings &"
2,"Infinix SMART 7 (Emerald Green, 64 GB) (4 GB ...","₹7,299",4.2,,"13,920 Ratings &"
3,"POCO C51 (Royal Blue, 64 GB) (4 GB RAM)","₹6,999",4.3,,"1,041 Ratings &"
4,"SAMSUNG Galaxy F13 (Nightsky Green, 64 GB) (4...","₹10,999",4.4,,"1,40,276 Ratings &"
...,...,...,...,...,...
19,GREENBERRI VIRAT (BLACK & BLUE),₹749,3.8,,507 Ratings &
20,"Infinix HOT 12 Play (Horizon Blue, 64 GB) (4 ...","₹8,999",4.3,,"1,15,557 Ratings &"
21,"Infinix HOT 12 Play (Racing Black, 64 GB) (4 ...","₹8,999",4.3,,"1,15,557 Ratings &"
22,"Infinix HOT 12 Play (Daylight Green, 64 GB) (...","₹8,999",4.3,,"1,15,557 Ratings &"


In [70]:
final_data1.shape

(936, 5)

In [65]:
# Save the data to a CSV file

final_data1.to_csv('Flipkart.csv', index=False)

## Insert scrapped data into Mongodb

In [66]:
dict1 = final_data1.to_dict(orient = "records")

In [67]:
dict1

[{'title': 'SAMSUNG Galaxy F04 (Opal Green, 64 GB)\xa0\xa0(4 GB RAM)',
  'price': '₹6,999',
  'rating': '4.3',
  'camera_rating': '',
  'rating_count': '10,820 Ratings &'},
 {'title': 'Infinix SMART 7 (Azure Blue, 64 GB)\xa0\xa0(4 GB RAM)',
  'price': '₹7,299',
  'rating': '4.2',
  'camera_rating': '',
  'rating_count': '13,920 Ratings &'},
 {'title': 'Infinix SMART 7 (Emerald Green, 64 GB)\xa0\xa0(4 GB RAM)',
  'price': '₹7,299',
  'rating': '4.2',
  'camera_rating': '',
  'rating_count': '13,920 Ratings &'},
 {'title': 'POCO C51 (Royal Blue, 64 GB)\xa0\xa0(4 GB RAM)',
  'price': '₹6,999',
  'rating': '4.3',
  'camera_rating': '',
  'rating_count': '1,041 Ratings &'},
 {'title': 'SAMSUNG Galaxy F13 (Nightsky Green, 64 GB)\xa0\xa0(4 GB RAM)',
  'price': '₹10,999',
  'rating': '4.4',
  'camera_rating': '',
  'rating_count': '1,40,276 Ratings &'},
 {'title': 'SAMSUNG Galaxy F13 (Waterfall Blue, 64 GB)\xa0\xa0(4 GB RAM)',
  'price': '₹10,999',
  'rating': '4.4',
  'camera_rating': '',
  '

In [68]:
db.Flipkart.insert_many(dict1)

## Insert Mobile_data into Mongodb


In [71]:
df2 = pd.read_csv("Mobile_data.csv")

In [72]:
df2.head()

,asin,brand,title,url,image,rating,reviewUrl,totalReviews,price,originalPrice
0,B0000SX2UC,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.00,0.0
1,B0009N5L7K,Motorola,Motorola I265 phone,https://www.amazon.com/Motorola-i265-I265-phon...,https://m.media-amazon.com/images/I/419WBAVDAR...,3.0,https://www.amazon.com/product-reviews/B0009N5L7K,7,49.95,0.0
2,B000SKTZ0S,Motorola,MOTOROLA C168i AT&T CINGULAR PREPAID GOPHONE C...,https://www.amazon.com/MOTOROLA-C168i-CINGULAR...,https://m.media-amazon.com/images/I/71b+q3ydkI...,2.7,https://www.amazon.com/product-reviews/B000SKTZ0S,22,99.99,0.0
3,B001AO4OUC,Motorola,Motorola i335 Cell Phone Boost Mobile,https://www.amazon.com/Motorola-i335-Phone-Boo...,https://m.media-amazon.com/images/I/710UO8gdT+...,3.3,https://www.amazon.com/product-reviews/B001AO4OUC,21,0.00,0.0
4,B001DCJAJG,Motorola,Motorola V365 no contract cellular phone AT&T,https://www.amazon.com/Motorola-V365-contract-...,https://m.media-amazon.com/images/I/61LYNCVrrK...,3.1,https://www.amazon.com/product-reviews/B001DCJAJG,12,149.99,0.0


In [73]:
dict2 = df2.to_dict(orient = "records")

In [74]:
dict2

[{'asin': 'B0000SX2UC',
  'brand': nan,
  'title': 'Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice Activated Dialing & Bright White Backlit Screen',
  'url': 'https://www.amazon.com/Dual-Band-Tri-Mode-Activated-Dialing-Backlit/dp/B0000SX2UC',
  'image': 'https://m.media-amazon.com/images/I/2143EBQ210L._AC_UY218_ML3_.jpg',
  'rating': 3.0,
  'reviewUrl': 'https://www.amazon.com/product-reviews/B0000SX2UC',
  'totalReviews': 14,
  'price': 0.0,
  'originalPrice': 0.0},
 {'asin': 'B0009N5L7K',
  'brand': 'Motorola',
  'title': 'Motorola I265 phone',
  'url': 'https://www.amazon.com/Motorola-i265-I265-phone/dp/B0009N5L7K',
  'image': 'https://m.media-amazon.com/images/I/419WBAVDARL._AC_UY218_ML3_.jpg',
  'rating': 3.0,
  'reviewUrl': 'https://www.amazon.com/product-reviews/B0009N5L7K',
  'totalReviews': 7,
  'price': 49.95,
  'originalPrice': 0.0},
 {'asin': 'B000SKTZ0S',
  'brand': 'Motorola',
  'title': 'MOTOROLA C168i AT&T CINGULAR PREPAID GOPHONE CELL PHONE',
  'url': 'https://www.amaz

In [ ]:
db.Mobile_data.insert_many(dict2)